# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data_file = "../output_data/cities.csv"
weather_df = pd.read_csv(weather_data_file, encoding = "UTF-8")
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Torbay,47.6666,-52.7314,33.01,86,90,32.21,CA,1612901282
1,Albany,42.6001,-73.9662,28.00,87,100,3.06,US,1612901116
2,Vaini,-21.2000,-175.2000,80.60,89,90,21.85,TO,1612901274
3,Thaba-Tseka,-29.5220,28.6084,57.85,72,100,1.10,LS,1612901544
4,Butaritari,3.0707,172.7902,80.89,83,5,16.55,KI,1612901267


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = weather_df["Humidity"]

In [5]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heatmap_layer = gmaps.heatmap_layer(locations, weights= humidity, 
                                 dissipating=False, max_intensity=100, point_radius=3.0)


# Add layer
fig.add_layer(heatmap_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Create new DataFrame with cities that meet specified weather condition parameters
ideal_weather_df = weather_df[(weather_df["Max Temp"]<75) & (weather_df['Max Temp']>68) & 
                              (weather_df['Humidity']<75) & (weather_df['Wind Speed']>10)]
ideal_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
43,Mar del Plata,-38.0023,-57.5575,69.80,68,75,16.11,AR,1612901178
48,Chui,-33.6971,-53.4616,73.63,72,0,10.07,UY,1612901266
65,Hilo,19.7297,-155.0900,73.40,68,90,11.50,US,1612901196
80,Kapaa,22.0752,-159.3190,71.60,49,1,21.85,US,1612901271
104,Ishigaki,24.3448,124.1572,68.13,74,99,18.70,JP,1612901551
206,Lüderitz,-26.6481,15.1594,69.80,68,0,12.46,NaN,1612901338
236,Dwārka,22.2394,68.9678,69.44,74,33,10.58,IN,1612901557
244,Kahului,20.8947,-156.4700,71.60,68,75,18.41,US,1612901331
288,Ormara,25.2088,64.6357,74.12,38,13,13.13,PK,1612901304
430,Praia,14.9215,-23.5087,69.80,73,0,23.02,CV,1612901212


In [7]:
# Drop any rows with null values
ideal_weather_df.dropna(inplace = True) 
ideal_weather_df

C:\Users\bisma\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
43,Mar del Plata,-38.0023,-57.5575,69.80,68,75,16.11,AR,1612901178
48,Chui,-33.6971,-53.4616,73.63,72,0,10.07,UY,1612901266
65,Hilo,19.7297,-155.0900,73.40,68,90,11.50,US,1612901196
80,Kapaa,22.0752,-159.3190,71.60,49,1,21.85,US,1612901271
104,Ishigaki,24.3448,124.1572,68.13,74,99,18.70,JP,1612901551
236,Dwārka,22.2394,68.9678,69.44,74,33,10.58,IN,1612901557
244,Kahului,20.8947,-156.4700,71.60,68,75,18.41,US,1612901331
288,Ormara,25.2088,64.6357,74.12,38,13,13.13,PK,1612901304
430,Praia,14.9215,-23.5087,69.80,73,0,23.02,CV,1612901212
455,Kailua,21.4022,-157.7394,73.40,53,40,17.27,US,1612901568


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = ideal_weather_df[['Country', 'City', 'Lat', 'Lng']]
hotel_df['Hotel Name']= ''
hotel_df

C:\Users\bisma\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Country,City,Lat,Lng,Hotel Name
43,AR,Mar del Plata,-38.0023,-57.5575,
48,UY,Chui,-33.6971,-53.4616,
65,US,Hilo,19.7297,-155.0900,
80,US,Kapaa,22.0752,-159.3190,
104,JP,Ishigaki,24.3448,124.1572,
236,IN,Dwārka,22.2394,68.9678,
244,US,Kahului,20.8947,-156.4700,
288,PK,Ormara,25.2088,64.6357,
430,CV,Praia,14.9215,-23.5087,
455,US,Kailua,21.4022,-157.7394,


In [9]:
# find the closest hotel to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get lat and Lng coordinates from df
    lat = row ['Lat']
    lng = row ['Lng']
    city = row ['City']

    # add keyword to params dict
    params['location'] = (f'{lat},{lng}')

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel in {city} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    print("-------------------------")

Closest hotel in Mar del Plata is Gran Hotel Mar del Plata.
-------------------------


C:\Users\bisma\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Closest hotel in Chui is Nuevo Hotel Plaza.
-------------------------
Closest hotel in Hilo is Hilo Hawaiian Hotel.
-------------------------
Closest hotel in Kapaa is Sheraton Kauai Resort at Coconut Beach.
-------------------------
Closest hotel in Ishigaki is Hotel Peace Land Ishigakijima.
-------------------------
Closest hotel in Dwārka is Hotel Narayan Avezika Comfort.
-------------------------
Closest hotel in Kahului is Maui Seaside Hotel.
-------------------------
Closest hotel in Ormara is Crystal Beach and Camping Resort Ormara.
-------------------------
Closest hotel in Praia is LT Aparthotel.
-------------------------
Closest hotel in Kailua is Sheffield House Bed & Breakfast.
-------------------------
Missing field/result... skipping.
-------------------------
Closest hotel in Copiapó is Antay Casino Hotel.
-------------------------


In [10]:
# Display DataFrame
hotel_df

,Country,City,Lat,Lng,Hotel Name
43,AR,Mar del Plata,-38.0023,-57.5575,Gran Hotel Mar del Plata
48,UY,Chui,-33.6971,-53.4616,Nuevo Hotel Plaza
65,US,Hilo,19.7297,-155.0900,Hilo Hawaiian Hotel
80,US,Kapaa,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
104,JP,Ishigaki,24.3448,124.1572,Hotel Peace Land Ishigakijima
236,IN,Dwārka,22.2394,68.9678,Hotel Narayan Avezika Comfort
244,US,Kahului,20.8947,-156.4700,Maui Seaside Hotel
288,PK,Ormara,25.2088,64.6357,Crystal Beach and Camping Resort Ormara
430,CV,Praia,14.9215,-23.5087,LT Aparthotel
455,US,Kailua,21.4022,-157.7394,Sheffield House Bed & Breakfast


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))